In [ ]:
import functions
import pandas as pd

The only preprocessing that is not handled in the funtions is the filtering of channels that will drown out other signals for the clustering

In [ ]:
# Paths to files
feature_tiff_path = '/data/raw_data/urothelial_carcinoma/urothelial_carcinoma_feature.tiff'
mask_tiff_path = '/data/raw_data/urothelial_carcinoma/urothelial_carcinoma_segmentation_mask.tiff'
channels_csv_path = '/data/raw_data/urothelial_carcinoma/urothelial_carcinoma_feature.csv'

channels_df = pd.read_csv(channels_csv_path)
mask = read_tiff_image(mask_tiff_path)

# Not including channels that will likely only lead to noise
#print(channels_df)
noisy_channels = {0,1,2,3,5,8,17,22,26,27,28,33,34,36,39,40,41,42}
# removed channels: 80ArAr (ArAr80), HistoneH3(In113), <EMPTY> (In115), 129Xe (Xe129), <EMPTY> (Nd142), <EMPTY> (Nd145),
# <EMPTY> (Sm154), <EMPTY> (Gd160), <EMPTY> (Dy164), <EMPTY> (Ho165), <EMPTY> (Er166), <EMPTY> (Yb171), <EMPTY> (Yb172),
# <EMPTY> (Yb174), 190BCKG (BCKG190), DNA1 (Ir191), DNA2 (Ir193), <EMPTY> (Pb204)

# Calculate remaining channels by excluding noisy ones
all_channels = set(range(len(channels_df)))
remaining_channels = list(all_channels - noisy_channels)

# Select the non-noisy channels
filtered_img_data = read_tiff_image(feature_tiff_path)[remaining_channels]
channels_df = channels_df.iloc[remaining_channels]

data, cell_ids, centroids, dataScaleSize, cellSizes = extract_data_from_tiffs(feature_tiff_path, mask, remaining_channels)
feature_img = read_tiff_image(feature_tiff_path)

The extracted data can then be used to either directly run UTAG

In [ ]:
unmod_adata = utag_create_anndata(feature_img, mask, channels_df)

utag_unmod = utag(
    unmod_adata,
    slide_key="roi",
    max_dist=15,
    normalization_mode='l1_norm',
    apply_clustering=True,
    clustering_method = 'parc',
    resolutions = [0.3]
)

Alternatively REDSEA spillover compensation can be applied before clustering with   

In [ ]:
# Running REDSEA
data_compen_scale_size_cells, means, stdevs, valid_indices = redsea_compensation(data, mask, filtered_img_data, channels_df, feature_tiff_path,
                                                                                 cell_sizes, element_shape=2, element_size=2)

redsea_adata = create_anndata(data_compen_scale_size_cells, cell_ids, centroids, means, stdevs, channels_df, valid_indices)
# Running UTAG
utag_redsea = utag(
    redsea_adata,
    slide_key="roi",
    max_dist=15,
    normalization_mode='l1_norm',
    apply_clustering=True,
    clustering_method = 'parc',
    resolutions = [0.3]
)